## Read data

In [1]:
import pandas as pd
import csv
import os
import numpy as np
import datetime
from xlsxwriter.workbook import Workbook
from xlsxwriter import Workbook
import time
Working_dir = os.getcwd()

In [2]:
zabbix_df = pd.read_excel( os.path.join( Working_dir, "zabbix_v2_init.xlsx" ) )

In [2]:
lsf_df = pd.read_excel( os.path.join( Working_dir, "ultimate_eleboration_job_cresco4.xlsx" ), header=None )

In [5]:
lsf_df_long_jobs = pd.read_excel( os.path.join( Working_dir, "lsf_v3_long_jobs_py.xlsx" ) )

In [4]:
lsf_df_short_long_jobs = pd.read_excel( os.path.join( Working_dir, "lsf_v3_short_and_long_jobs_py.xlsx" ) )

Create Python date and time in comparable format (UTC)

In [4]:
print(zabbix_df.columns.values)
print(lsf_df.columns.values)

['Start_Timestamp (1h interval)' 'min_power' 'avg_power' 'max_power'
 'measure time in seconds']
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]


In [5]:
lsf_df.columns =(['row_counter',\
                         'job_id',\
                         'number_of_cores_used',\
                         'user_name',\
                         'queue_len_hours',\
                         'queue_name',\
                         'directory_of_running_executable',\
                         'name_of_executable',\
                         'job_status',\
                         'start_unixtime',\
                         'stop_unixtime',\
                         'job_duration_sec',\
                         'queue_duration_sec',\
                         'percentage_of_usage_of_queue',\
                         'residual_time_of_queue',\
                         'application_performed_and_scope'])

In [6]:
print(lsf_df.columns.values)

['row_counter' 'job_id' 'number_of_cores_used' 'user_name'
 'queue_len_hours' 'queue_name' 'directory_of_running_executable'
 'name_of_executable' 'job_status' 'start_unixtime' 'stop_unixtime'
 'job_duration_sec' 'queue_duration_sec' 'percentage_of_usage_of_queue'
 'residual_time_of_queue' 'application_performed_and_scope']


In [7]:
zabbix_df["date_py"] = zabbix_df["Start_Timestamp (1h interval)"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).date())
zabbix_df["time_py"] = zabbix_df["Start_Timestamp (1h interval)"].apply(lambda x: datetime.datetime.utcfromtimestamp(x).time())
zabbix_df["timestamp_py"] = zabbix_df["Start_Timestamp (1h interval)"].apply(lambda x: datetime.datetime.utcfromtimestamp(x))

In [8]:
lsf_df["start_timestamp_py"] = lsf_df["start_unixtime"].apply(lambda x: datetime.datetime.utcfromtimestamp(x))
lsf_df["stop_timestamp_py"] = lsf_df["stop_unixtime"].apply(lambda x: datetime.datetime.utcfromtimestamp(x))
lsf_df['directory_of_running_executable'] = lsf_df['directory_of_running_executable'].fillna('')
lsf_df['name_of_executable'] = lsf_df['name_of_executable'].fillna('')

In [9]:
lsf_df['executable_full_name'] = lsf_df['directory_of_running_executable'].apply(lambda x: \
                                                                                            str(x) if type(x) == int else x ) + \
lsf_df["name_of_executable"].apply( lambda x: str(x) if type(x) == int else \
                                    (x.strftime('%H-%M-%S') if type(x)==datetime.time else x) )

In [10]:
zabbix_df['energy_avg (kWh)']=zabbix_df['avg_power'].copy()*zabbix_df['measure time in seconds'].copy()/60000.
zabbix_df['energy_min (kWh)']=zabbix_df['min_power'].copy()*zabbix_df['measure time in seconds'].copy()/60000.
zabbix_df['energy_max (kWh)']=zabbix_df['max_power'].copy()*zabbix_df['measure time in seconds'].copy()/60000.

In [11]:
zabbix_df.head()

,Start_Timestamp (1h interval),min_power,avg_power,max_power,measure time in seconds,date_py,time_py,timestamp_py,energy_avg (kWh),energy_min (kWh),energy_max (kWh)
0,1487505600,38600,40637,41500,60.00,2017-02-19,12:00:00,2017-02-19 12:00:00,40.637000,38.600000,41.500000
1,1487509200,40010,40863,41530,60.00,2017-02-19,13:00:00,2017-02-19 13:00:00,40.863000,40.010000,41.530000
2,1487512800,37740,39971,41430,59.75,2017-02-19,14:00:00,2017-02-19 14:00:00,39.804454,37.582750,41.257375
3,1487516400,36190,38517,39470,60.25,2017-02-19,15:00:00,2017-02-19 15:00:00,38.677487,36.340792,39.634458
4,1487520000,38330,38735,39200,60.00,2017-02-19,16:00:00,2017-02-19 16:00:00,38.735000,38.330000,39.200000


#### Different processes names can be identifiers, so we form the full names in lsf table 
Number of different job names ("folder+name")

In [12]:
print('Unique names of jobs in a year: ',len(lsf_df.executable_full_name.unique()))
print('Jobs done in a year (including repetitions of the same jobs): ', lsf_df.shape[0])

Unique names of jobs in a year:  11530
Jobs done in a year (including repetitions of the same jobs):  529512


User names

In [13]:
print('Unique user names in a year: ',len(lsf_df['user_name'].unique()))

Unique user names in a year:  119


In [14]:
lsf_df.head()

,row_counter,job_id,number_of_cores_used,user_name,queue_len_hours,queue_name,directory_of_running_executable,name_of_executable,job_status,start_unixtime,stop_unixtime,job_duration_sec,queue_duration_sec,percentage_of_usage_of_queue,residual_time_of_queue,application_performed_and_scope,start_timestamp_py,stop_timestamp_py,executable_full_name
0,1,967084,32,gusso,6.0,cresco4_h6,PFS/porq1_1M/SiH/Grossman/cp2k/asi561/UnCharge...,cp2k_launch.sh,DONE,1483190848,1.483212e+09,21245.0,21600.0,98.356481,355.0,"CP2K atomistic simulations of solid state, liq...",2016-12-31 13:27:28,2016-12-31 19:21:33,PFS/porq1_1M/SiH/Grossman/cp2k/asi561/UnCharge...
1,2,967085,32,gusso,6.0,cresco4_h6,PFS/porq1_1M/SiH/Grossman/cp2k/asi561/UnCharge...,cp2k_launch.sh,DONE,1483190848,1.483212e+09,21286.0,21600.0,98.546296,314.0,"CP2K atomistic simulations of solid state, liq...",2016-12-31 13:27:28,2016-12-31 19:22:14,PFS/porq1_1M/SiH/Grossman/cp2k/asi561/UnCharge...
2,3,967086,32,gusso,6.0,cresco4_h6,PFS/porq1_1M/SiH/Grossman/cp2k/asi561/UnCharge...,cp2k_launch.sh,DONE,1483190848,1.483212e+09,21145.0,21600.0,97.893519,455.0,"CP2K atomistic simulations of solid state, liq...",2016-12-31 13:27:28,2016-12-31 19:19:53,PFS/porq1_1M/SiH/Grossman/cp2k/asi561/UnCharge...
3,4,967087,32,gusso,6.0,cresco4_h6,PFS/porq1_1M/SiH/Grossman/cp2k/asi561/UnCharge...,cp2k_launch.sh,DONE,1483190848,1.483212e+09,21150.0,21600.0,97.916667,450.0,"CP2K atomistic simulations of solid state, liq...",2016-12-31 13:27:28,2016-12-31 19:19:58,PFS/porq1_1M/SiH/Grossman/cp2k/asi561/UnCharge...
4,5,967088,32,gusso,6.0,cresco4_h6,PFS/porq1_1M/SiH/Grossman/cp2k/asi561/UnCharge...,cp2k_launch.sh,DONE,1483190848,1.483212e+09,21286.0,21600.0,98.546296,314.0,"CP2K atomistic simulations of solid state, liq...",2016-12-31 13:27:28,2016-12-31 19:22:14,PFS/porq1_1M/SiH/Grossman/cp2k/asi561/UnCharge...


In [15]:
print(zabbix_df.columns.values, "\n")
print(lsf_df.columns.values)

['Start_Timestamp (1h interval)' 'min_power' 'avg_power' 'max_power'
 'measure time in seconds' 'date_py' 'time_py' 'timestamp_py'
 'energy_avg (kWh)' 'energy_min (kWh)' 'energy_max (kWh)'] 

['row_counter' 'job_id' 'number_of_cores_used' 'user_name'
 'queue_len_hours' 'queue_name' 'directory_of_running_executable'
 'name_of_executable' 'job_status' 'start_unixtime' 'stop_unixtime'
 'job_duration_sec' 'queue_duration_sec' 'percentage_of_usage_of_queue'
 'residual_time_of_queue' 'application_performed_and_scope'
 'start_timestamp_py' 'stop_timestamp_py' 'executable_full_name']


In [16]:
print(zabbix_df.shape)
print(lsf_df.shape)

(8763, 11)
(529512, 19)


In [17]:
excel_writer_zabbix = pd.ExcelWriter( os.path.join( Working_dir, "zabbix_v3_py.xlsx"))
zabbix_df.to_excel(excel_writer_zabbix)
excel_writer_zabbix.save()

In [18]:
lsf_df = lsf_df.sort_values('stop_timestamp_py')

In [19]:
excel_writer_lsf = pd.ExcelWriter( os.path.join( Working_dir, "lsf_v3_py.xlsx" ))
lsf_df.to_excel(excel_writer_lsf)
excel_writer_lsf.save()

-----

### Queue - long jobs
### Preprocessing for the analysis of jobs which quitted their queues with error beacause they were running longer than the queue permitted
- Create lsf_v3.1 where all jobs which were running longer than their queue duration will be separated into two lines with corresponding statuses "done_long" and "exit_long". 
- The first will be reported to work within the queue duration and produce useful work, the second will refer to the processing of the job which exceeded queue time. 
- Start and stop time moments will be updated as well as the percentage of the queue usage.

In [21]:
lsf_df_in_case = lsf_df.copy()

In [22]:
lsf_df_long_jobs = lsf_df.copy()

In [109]:
long_jobs_ix_0 = lsf_df[lsf_df.job_duration_sec > lsf_df.queue_duration_sec].index
long_jobs_ix_1 = lsf_df_long_jobs[lsf_df_long_jobs.job_duration_sec == lsf_df_long_jobs.queue_duration_sec].index

In [120]:
i=0
while not(lsf_df_long_jobs[lsf_df_long_jobs.job_duration_sec > lsf_df_long_jobs.queue_duration_sec].empty):
    index = lsf_df_long_jobs[lsf_df_long_jobs.job_duration_sec > lsf_df_long_jobs.queue_duration_sec].index[0]
    line = lsf_df_long_jobs.loc[index]

    line_done = line.copy()
    line_exit = line.copy()

    line_done.job_status = 'done_long'
    line_exit.job_status = 'exit_long'
    
    line_done.job_duration_sec = line.queue_duration_sec
    line_exit.job_duration_sec = line.job_duration_sec - line.queue_duration_sec

    line_done.stop_unixtime = line.start_unixtime + line_done.job_duration_sec
    line_done.stop_timestamp_py = datetime.datetime.utcfromtimestamp(line_done.stop_unixtime)

    line_exit.start_unixtime = line_done.stop_unixtime
    line_exit.start_timestamp_py = datetime.datetime.utcfromtimestamp(line_exit.start_unixtime)

    line_done.residual_time_of_queue = 0
    line_exit.residual_time_of_queue = None
    
    line_done.percentage_of_usage_of_queue = 100.
    line_exit.percentage_of_usage_of_queue  = line.percentage_of_usage_of_queue - 100.

    lsf_df_long_jobs.loc[index] = line_done

    lsf_df_long_jobs = lsf_df_long_jobs.append(line_exit)
    i+=1
    if i%100==0:
        print(i)
    if i>lsf_df.shape[0]:
        break

100
200
300
400
500
600
700
800
900
1000
1100


In [121]:
i

1156

In [124]:
print(lsf_df.shape[0] + len(long_jobs_ix_0), lsf_df_long_jobs.shape[0], lsf_df.shape[0], len(long_jobs_ix_0))

530668 530668 529512 1156


In [125]:
excel_writer_lsf = pd.ExcelWriter( os.path.join( Working_dir, "lsf_v3_long_jobs_py.xlsx" ))
lsf_df_long_jobs.to_excel(excel_writer_lsf)
excel_writer_lsf.save()

------
### Short jobs, <=30 sec
- Add marker "exit_short" to job_status column for short jobs

In [126]:
lsf_df_short_long_jobs = lsf_df_long_jobs.copy()

In [127]:
index_short = lsf_df_short_long_jobs[(lsf_df_short_long_jobs.job_duration_sec <= 30)\
                                    &(lsf_df_short_long_jobs.job_status!='exit_long')].index

lsf_df_short_long_jobs.loc[index_short, 'job_status'] = "exit_short_30_sec"

In [91]:
lsf_df_short_long_jobs[lsf_df_short_long_jobs.job_status=='exit_short_30_sec'].shape

(101843, 19)

In [128]:
excel_writer_lsf = pd.ExcelWriter( os.path.join( Working_dir, "lsf_v3_short_and_long_jobs_py.xlsx" ))
lsf_df_short_long_jobs.to_excel(excel_writer_lsf)
excel_writer_lsf.save()

---------------

### Sorting values, updating indices to be unique and saving for division into months

In [36]:
lsf_df_long_jobs_s = lsf_df_long_jobs.copy()
lsf_df_short_long_jobs_s = lsf_df_short_long_jobs.copy()

In [37]:
lsf_df_long_jobs_s.sort_index(inplace=True)
lsf_df_short_long_jobs_s.sort_index(inplace=True)

lsf_df_long_jobs_s[ (lsf_df_long_jobs_s.job_status=='exit_long') | (lsf_df_long_jobs_s.job_status=='done_long') ].head()

,row_counter,job_id,number_of_cores_used,user_name,queue_len_hours,queue_name,directory_of_running_executable,name_of_executable,job_status,start_unixtime,stop_unixtime,job_duration_sec,queue_duration_sec,percentage_of_usage_of_queue,residual_time_of_queue,application_performed_and_scope,start_timestamp_py,stop_timestamp_py,executable_full_name
583,646,953307,12,aprea,144.0,cresco4_h144,NaN,./assembly.sh,done_long,1.478886e+09,1.479404e+09,518400.0,518400.0,100.000000,0.0,genetic analysis,2016-11-11 17:38:05,2016-11-17 17:38:05,./assembly.sh
583,646,953307,12,aprea,144.0,cresco4_h144,NaN,./assembly.sh,exit_long,1.479404e+09,1.479405e+09,586.0,518400.0,0.113040,NaN,genetic analysis,2016-11-17 17:38:05,2016-11-17 17:47:51,./assembly.sh
584,647,953308,12,aprea,144.0,cresco4_h144,NaN,./assembly.sh,done_long,1.478886e+09,1.479404e+09,518400.0,518400.0,100.000000,0.0,genetic analysis,2016-11-11 17:38:05,2016-11-17 17:38:05,./assembly.sh
584,647,953308,12,aprea,144.0,cresco4_h144,NaN,./assembly.sh,exit_long,1.479404e+09,1.479405e+09,506.0,518400.0,0.097608,NaN,genetic analysis,2016-11-17 17:38:05,2016-11-17 17:46:31,./assembly.sh
751,868,967910,32,gusso,6.0,cresco4_h6,PFS/porq1_1M/SiH/Grossman/cp2k/asi632/UnCharge...,cp2k_launch.sh,exit_long,1.483320e+09,1.483321e+09,539.0,21600.0,2.495370,NaN,"CP2K atomistic simulations of solid state, liq...",2017-01-02 01:26:35,2017-01-02 01:35:34,PFS/porq1_1M/SiH/Grossman/cp2k/asi632/UnCharge...


In [38]:
lsf_df_long_jobs_s.reset_index(inplace=True, drop=True)
lsf_df_short_long_jobs_s.reset_index(inplace=True, drop=True)

lsf_df_long_jobs_s[ (lsf_df_long_jobs_s.job_status=='exit_long') | (lsf_df_long_jobs_s.job_status=='done_long') ].head()

,row_counter,job_id,number_of_cores_used,user_name,queue_len_hours,queue_name,directory_of_running_executable,name_of_executable,job_status,start_unixtime,stop_unixtime,job_duration_sec,queue_duration_sec,percentage_of_usage_of_queue,residual_time_of_queue,application_performed_and_scope,start_timestamp_py,stop_timestamp_py,executable_full_name
583,646,953307,12,aprea,144.0,cresco4_h144,NaN,./assembly.sh,done_long,1.478886e+09,1.479404e+09,518400.0,518400.0,100.000000,0.0,genetic analysis,2016-11-11 17:38:05,2016-11-17 17:38:05,./assembly.sh
584,646,953307,12,aprea,144.0,cresco4_h144,NaN,./assembly.sh,exit_long,1.479404e+09,1.479405e+09,586.0,518400.0,0.113040,NaN,genetic analysis,2016-11-17 17:38:05,2016-11-17 17:47:51,./assembly.sh
585,647,953308,12,aprea,144.0,cresco4_h144,NaN,./assembly.sh,done_long,1.478886e+09,1.479404e+09,518400.0,518400.0,100.000000,0.0,genetic analysis,2016-11-11 17:38:05,2016-11-17 17:38:05,./assembly.sh
586,647,953308,12,aprea,144.0,cresco4_h144,NaN,./assembly.sh,exit_long,1.479404e+09,1.479405e+09,506.0,518400.0,0.097608,NaN,genetic analysis,2016-11-17 17:38:05,2016-11-17 17:46:31,./assembly.sh
753,868,967910,32,gusso,6.0,cresco4_h6,PFS/porq1_1M/SiH/Grossman/cp2k/asi632/UnCharge...,cp2k_launch.sh,exit_long,1.483320e+09,1.483321e+09,539.0,21600.0,2.495370,NaN,"CP2K atomistic simulations of solid state, liq...",2017-01-02 01:26:35,2017-01-02 01:35:34,PFS/porq1_1M/SiH/Grossman/cp2k/asi632/UnCharge...


In [39]:
print(lsf_df_long_jobs_s.shape, lsf_df_short_long_jobs_s.shape)

(530668, 19) (530668, 19)


In [40]:
lsf_df_long_jobs_s.sort_values('stop_timestamp_py', inplace=True)
lsf_df_short_long_jobs_s.sort_values('stop_timestamp_py', inplace=True)

In [41]:
excel_writer_lsf = pd.ExcelWriter( os.path.join( Working_dir, "lsf_v3_short_and_long_jobs_sorted_py.xlsx" ))
lsf_df_short_long_jobs_s.to_excel(excel_writer_lsf)
excel_writer_lsf.save()

In [34]:
excel_writer_lsf = pd.ExcelWriter( os.path.join( Working_dir, "lsf_v3_long_jobs_sorted_py.xlsx" ))
lsf_df_long_jobs_s.to_excel(excel_writer_lsf)
excel_writer_lsf.save()

### Check date ranges for zabbix and new lsf file (version 3)

In [22]:
print('lsf range: ', lsf_df.start_timestamp_py.min(), lsf_df.start_timestamp_py.max())
print('zabbix range: ', zabbix_df.timestamp_py.min(), zabbix_df.timestamp_py.max())

lsf range:  2016-11-11 17:38:05 2018-01-25 04:07:31
zabbix range:  2017-02-19 12:00:00 2018-02-19 14:00:00


### Create new column in "lsf_v3_short_and_long_jobs_sorted_py": parallel / serial jobs marker

In [2]:
lsf_df = pd.read_excel( os.path.join( Working_dir, "lsf_v3_short_and_long_jobs_sorted_py.xlsx" ) )

In [7]:
# lsf_df_in_case = lsf_df.copy()

In [10]:
lsf_df['parallel_serial'] = lsf_df['number_of_cores_used'].apply( lambda x: 'serial' if x==1 else 'parallel' )

#### Number of jobs that were executed as parallel and serial within the whole time period. 

In [35]:
a = set(lsf_df[lsf_df.parallel_serial=='parallel']['executable_full_name'].unique())
b = set(lsf_df[lsf_df.parallel_serial=='serial']['executable_full_name'].unique())
print(len(a.intersection(b)))

111


#### Drop unnecessary jobs which do not affect 19.02.2017 - 25.01.2018

In [27]:
left_time_border = datetime.datetime(2017,2,19,12,0)

lsf_df = lsf_df.drop(lsf_df[lsf_df.stop_timestamp_py < left_time_border].index )
print('lsf range: ', lsf_df.start_timestamp_py.min(), lsf_df.start_timestamp_py.max())

lsf range:  2017-02-16 09:48:21 2018-01-25 04:07:31


In [38]:
print(lsf_df[lsf_df.parallel_serial=='parallel'].shape, lsf_df[lsf_df.parallel_serial=='parallel']['executable_full_name'].unique().shape)

(38816, 20) (8006,)


In [28]:
excel_writer_lsf = pd.ExcelWriter( os.path.join( Working_dir, "lsf_v3_short_and_long_jobs_sorted_py.xlsx" ))
lsf_df.to_excel(excel_writer_lsf)
excel_writer_lsf.save()